In [1]:
from datasets import load_dataset

ds = load_dataset("jtatman/python-code-dataset-500k")

/home/tanishq/Code/TanCode/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 559515/559515 [00:01<00:00, 370400.02 examples/s]


In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'system'],
        num_rows: 559515
    })
})

In [ ]:
print(ds["train"][0]['output'])

Here is an example of a nested loop in Python to print every combination of numbers between 0-9, excluding any combination that contains the number 5 or repeating digits:

```python
for i in range(10):  # First digit
    for j in range(10):  # Second digit
        for k in range(10):  # Third digit
            # Checking for the conditions
            if i != 5 and j != 5 and k != 5 and i != j and i != k and j != k:
                print(i, j, k)
```

This code will generate and print every combination of three digits between 0-9 that do not contain the number 5 and do not have any repeating digits.


## Checking Numbers of Unique Values

In [13]:
len(ds["train"].unique("output"))

493723

In [14]:
len(ds["train"].unique("instruction"))

494396

In [15]:
len(ds["train"].unique("system"))

27720

## Checking missing Values(Null/None)

In [16]:
for col in ds["train"].column_names:
    missing = ds["train"].filter(lambda x: x[col] is None)
    print(col, len(missing))

Filter: 100%|██████████| 559515/559515 [00:01<00:00, 288396.99 examples/s]


output 0


Filter: 100%|██████████| 559515/559515 [00:01<00:00, 292999.72 examples/s]


instruction 0


Filter: 100%|██████████| 559515/559515 [00:01<00:00, 294197.71 examples/s]

system 0


## Checking Empty Strings ("")

In [ ]:
for col in ds["train"].column_names:
    missing = ds["train"].filter(lambda x: x[col]== "")
    print(col, len(missing))

Filter: 100%|██████████| 559515/559515 [00:01<00:00, 296153.31 examples/s]


output 0


Filter: 100%|██████████| 559515/559515 [00:01<00:00, 294209.07 examples/s]


instruction 0


Filter: 100%|██████████| 559515/559515 [00:01<00:00, 290345.63 examples/s]

system 22608


## Checking Duplicate Values

In [ ]:
rows = ds["train"].to_pandas()
dup_count = rows.duplicated().sum()
print(dup_count)

26076


### Filing the empty strings in the system prompt 

In [21]:
def fill_system(example):
    value = example["system"]

    if (
        value is None or
        (isinstance(value, str) and value.strip() == "")
    ):
        example["system"] = "You are a helpful coding assistant."

    return example


ds["train"] = ds["train"].map(fill_system)


Map: 100%|██████████| 559515/559515 [00:23<00:00, 23686.06 examples/s]


## Remove Duplicate Values

In [ ]:
df = ds["train"].to_pandas()
df = df.drop_duplicates()
ds["train"] = ds["train"].from_pandas(df, preserve_index=False)

## Filter Outputs which have less than characters

In [25]:
ds["train"] = ds["train"].filter(
    lambda x: len(x["output"].strip()) > 5
)

Filter: 100%|██████████| 533439/533439 [00:02<00:00, 264534.63 examples/s]


In [ ]:
def clean(example):
    for k in ds["train"].column_names:
        if k in example and isinstance(example[k], str):
            example[k] = example[k].strip()
    return example

ds["train"] = ds["train"].map(clean)
